In [53]:
from util import test_basic
from demo_pb2 import *
import demo_pb2_grpc
import uuid
import money 

## Here are the schemas for rpc methods / messages
```
// -------------Checkout service-----------------

service CheckoutService {
    rpc PlaceOrder(PlaceOrderRequest) returns (PlaceOrderResponse) {}
}

message PlaceOrderRequest {
    string user_id = 1;
    string user_currency = 2;

    Address address = 3;
    string email = 5;
    CreditCardInfo credit_card = 6;
}

message PlaceOrderResponse {
    OrderResult order = 1;
}

// -------------Money ---------------
message Money {
    // The 3-letter currency code defined in ISO 4217.
    string currency_code = 1;

    // The whole units of the amount.
    // For example if `currencyCode` is `"USD"`, then 1 unit is one US dollar.
    int64 units = 2;

    // Number of nano (10^-9) units of the amount.
    // The value must be between -999,999,999 and +999,999,999 inclusive.
    // If `units` is positive, `nanos` must be positive or zero.
    // If `units` is zero, `nanos` can be positive, zero, or negative.
    // If `units` is negative, `nanos` must be negative or zero.
    // For example $-1.75 is represented as `units`=-1 and `nanos`=-750,000,000.
    int32 nanos = 3;
}


// ---------------Shipping Service----------

service ShippingService {
    rpc GetQuote(GetQuoteRequest) returns (GetQuoteResponse) {}
    rpc ShipOrder(ShipOrderRequest) returns (ShipOrderResponse) {}
}

message GetQuoteRequest {
    Address address = 1;
    repeated CartItem items = 2;
}

message GetQuoteResponse {
    Money cost_usd = 1;
}

message ShipOrderRequest {
    Address address = 1;
    repeated CartItem items = 2;
}

message ShipOrderResponse {
    string tracking_id = 1;
}

message Address {
    string street_address = 1;
    string city = 2;
    string state = 3;
    string country = 4;
    int32 zip_code = 5;
}

// -------------Payment service-----------------

service PaymentService {
    rpc Charge(ChargeRequest) returns (ChargeResponse) {}
}

message CreditCardInfo {
    string credit_card_number = 1;
    int32 credit_card_cvv = 2;
    int32 credit_card_expiration_year = 3;
    int32 credit_card_expiration_month = 4;
}

message ChargeRequest {
    Money amount = 1;
    CreditCardInfo credit_card = 2;
}

message ChargeResponse {
    string transaction_id = 1;
}
```

In [54]:
class CheckoutService(demo_pb2_grpc.CheckoutServiceServicer):
    def PlaceOrder(self, request, context):
        import grpc
        import uuid
        shipping_channel = grpc.insecure_channel('localhost:50052')
        shipping_stub = demo_pb2_grpc.ShippingServiceStub(shipping_channel)
        payment_channel = grpc.insecure_channel('localhost:50053')
        payment_stub = demo_pb2_grpc.PaymentServiceStub(payment_channel)
        # mock data to simulate getting from cart service
        items=[
            demo_pb2.CartItem(
                product_id="P1",
                quantity=2,
            ),
            demo_pb2.CartItem(
                product_id="P2",
                quantity=1,
            )
        ]
        shipping_request = demo_pb2.GetQuoteRequest(
            address=request.address,
            items=items
        )
        shipping_response = shipping_stub.GetQuote(shipping_request)
        total_cost = demo_pb2.Money(currency_code = "USD",units = 20 + shipping_response.cost_usd.units, nanos = shipping_response.cost_usd.nanos) 

        payment_request = demo_pb2.ChargeRequest(
            amount=total_cost,
            credit_card=request.credit_card
        )
        payment_response = payment_stub.Charge(payment_request)
        ship_order_request = demo_pb2.ShipOrderRequest(address=request.address, items=items)
        ship_order_response = shipping_stub.ShipOrder(ship_order_request)
        order_items = [demo_pb2.OrderItem(item=i, cost=demo_pb2.Money(units=9, nanos=990000000)) for i in items] # mock data for cost to simulate getting price from catalog
        order_result = demo_pb2.OrderResult(
            order_id=str(uuid.uuid4()), 
            shipping_tracking_id=ship_order_response.tracking_id,
            shipping_cost=shipping_response.cost_usd,
            shipping_address=request.address,
            items=order_items
        )

        place_order_response = demo_pb2.PlaceOrderResponse(order=order_result)

        return place_order_response
        
class ShippingService(demo_pb2_grpc.ShippingServiceServicer):
    def GetQuote(self, request, context):
        return demo_pb2.GetQuoteResponse(cost_usd = demo_pb2.Money(currency_code="USD", units=7, nanos=990000000))
    
    def ShipOrder(self, request, context):
        import uuid
        return demo_pb2.ShipOrderResponse(tracking_id = str(uuid.uuid4()))

class PaymentService(demo_pb2_grpc.PaymentServiceServicer):
    def Charge(self, request, context):
        import uuid
        return demo_pb2.ChargeResponse(transaction_id = str(uuid.uuid4()))

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (3757987137.py, line 34)

In [55]:
test_basic({CheckoutService: 50051, ShippingService: 50052, PaymentService: 50053})

b'Order ID: dae90180-9490-4b46-b422-d3ede6df3174\nShipping Tracking ID: 35752a6f-d151-4367-bbf2-5027dcf4ff40\nShipping Cost (USD): currency_code: "USD"\nunits: 7\nnanos: 990000000\n\nShipping Address: street_address: "123 Main Street"\ncity: "Berkeley"\nstate: "CA"\ncountry: "USA"\nzip_code: 94704\n\nOrdered Items:\nProduct ID: P1, Quantity: 2, Cost (USD): units: 9\nnanos: 990000000\n\nProduct ID: P2, Quantity: 1, Cost (USD): units: 9\nnanos: 990000000\n\n' b''
b'Server started on port 50051\nterminating server\n' b''
b'Server started on port 50052\nterminating server\n' b''
b'Server started on port 50053\nterminating server\n' b''
